# TCGA-UCEC – Notebook 0A
## Acquisition des données brutes (GDC/TCGA)

**Objectif** : Télécharger les données de comptage STAR et les métadonnées cliniques via l'API du GDC en utilisant l'interopérabilité R-Python. Cette étape garantit la provenance et la reproductibilité des données sources.

## 1. Configuration et Environnement

In [ ]:
# ==========================================================================================================
import os                                       # Navigation fichiers (DIRS, chemins relatifs)
import sys                                      # Interaction système
import pandas as pd                             # Manipulation de tables
import subprocess                               # Exécution de commandes avec streaming de sortie

from datetime import datetime

# ==========================================================================================================
# Définition de la racine du projet
PROJECT_ROOT = os.getcwd()

DIRS = {
    "RAW":       os.path.join(PROJECT_ROOT, "data", "raw"),
    "SCRIPTS":   os.path.join(PROJECT_ROOT, "scripts"),
    "PROCESSED": os.path.join(PROJECT_ROOT, "data", "processed")
}

for path in DIRS.values():
    os.makedirs(path, exist_ok=True)

# ==========================================================================================================
print(f"✅ Environnement configuré. Racine : {PROJECT_ROOT}")
print(f"✅ Dossiers d'acquisition prêts.")

✅ Environnement configuré. Racine : c:\Z\M2_AIDA\TCGA_UCEC_project
✅ Dossiers d'acquisition prêts.


## 2. Exécution du script d'acquisition (Rscript)

Le téléchargement s'appuie sur la librairie R **TCGAbiolinks**. Nous appelons le script fourni directement depuis ce notebook pour maintenir une traçabilité complète.

In [ ]:
# ==========================================================================================================
# Chemin vers l'exécutable R et le script moteur
R_EXECUTABLE = r"C:\Program Files\R\R-4.5.2\bin\x64\Rscript.exe"
r_script_path = os.path.join(DIRS["SCRIPTS"], "Script_data_download.R")

# Vérification du script R
if not os.path.exists(r_script_path):
    raise FileNotFoundError(f"❌ Script R introuvable : {r_script_path}")

# Vérification de Rscript
if not os.path.exists(R_EXECUTABLE):
    raise FileNotFoundError(f"❌ Rscript introuvable : {R_EXECUTABLE}")

print("⏳ Acquisition en cours : téléchargement depuis le GDC (cela peut prendre 5-10 minutes)...\n")

try:
    process = subprocess.Popen(
        [R_EXECUTABLE, r_script_path], 
        stdout=subprocess.PIPE, 
        stderr=subprocess.STDOUT, 
        text=True,
        encoding='utf-8'
    )

    # Filtrage des lignes de progression répétitives
    for line in process.stdout:
        # Ignorer lignes "Downloading:" et "remaining" lors du téléchargement
        if "Downloading:" in line or "remaining" in line:
            continue
        print(line, end='')

    return_code = process.wait()
    
    if return_code != 0:
        raise RuntimeError(f"❌ Script R échoué (code {return_code})")
    
    print("\n✅ Script R terminé avec succès.")

except Exception as e:
    print(f"\n❌ ERREUR : {e}")
    raise

# Validation des fichiers générés
required_files = ["counts_samples_x_genes.csv.gz", 
                  "metadata_clinical_merged.csv.gz",
                  "gene_annotation.csv.gz"]

print("\n" + "="*60)
print("VALIDATION DES FICHIERS")
print("="*60)

for fname in required_files:
    fpath = os.path.join(DIRS["RAW"], fname)
    if not os.path.exists(fpath):
        raise FileNotFoundError(f"❌ Fichier manquant : {fname}")
    size_mb = os.path.getsize(fpath) / (1024**2)
    print(f"✅ {fname:40} {size_mb:8.2f} MB")

print("\n✅ Acquisition complète et validée.")

⏳ Acquisition en cours : téléchargement depuis le GDC (cela peut prendre 5-10 minutes)...

Le chargement a nécessité le package : MatrixGenerics
Le chargement a nécessité le package : matrixStats

Attachement du package : 'MatrixGenerics'

Les objets suivants sont masqués depuis 'package:matrixStats':

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2,

## 2Bis. Relocalisation des fichiers (Workaround Post-Acquisition)

Le script R exporte par défaut dans un dossier nommé tcga_ucec_counts_export. Nous déplaçons ces fichiers vers data/raw/ pour respecter l'arborescence du projet et permettre aux notebooks suivants d'accéder aux données.

In [ ]:
import shutil

# Chemins source et cible (os est déjà importé en cellule 1)
source_dir = os.path.join(PROJECT_ROOT, "tcga_ucec_counts_export")
target_dir = DIRS["RAW"]

files_to_move = [
    "counts_samples_x_genes.csv.gz",
    "metadata_clinical_merged.csv.gz",
    "gene_annotation.csv.gz"
]

if os.path.exists(source_dir):
    print(f"📂 Dossier source détecté : {source_dir}")
    moved_count = 0

    for f in files_to_move:
        src_path = os.path.join(source_dir, f)
        dst_path = os.path.join(target_dir, f)
        
        if os.path.exists(src_path):
            shutil.move(src_path, dst_path)
            print(f"✅ Déplacé : {f}")
            moved_count += 1
            
    if moved_count > 0:
        print(f"\n✅ Harmonisation terminée. {moved_count} fichiers déplacés vers {target_dir}.")
else:
    # Vérification de sécurité si les fichiers sont déjà là (évite de planter au re-run)
    if all(os.path.exists(os.path.join(target_dir, f)) for f in files_to_move):
        print("✅ Les fichiers sont déjà correctement placés dans data/raw/.")
    else:
        print(f"❌ Erreur : Dossier source {source_dir} introuvable et fichiers absents de {target_dir}.")

📂 Dossier source détecté : c:\Z\M2_AIDA\TCGA_UCEC_project\tcga_ucec_counts_export
✅ Déplacé : counts_samples_x_genes.csv.gz
✅ Déplacé : metadata_clinical_merged.csv.gz
✅ Déplacé : gene_annotation.csv.gz

✅ Harmonisation terminée. 3 fichiers déplacés vers c:\Z\M2_AIDA\TCGA_UCEC_project\data\raw.


## 3. Vérification des fichiers téléchargés

Contrôle de la présence physique des fichiers dans `data/raw/` et affichage des dimensions brutes.

In [9]:
# ==========================================================================================================
# Définition des chemins vers les fichiers attendus dans data/raw/
expr_file = os.path.join(DIRS["RAW"], "counts_samples_x_genes.csv.gz")
meta_file = os.path.join(DIRS["RAW"], "metadata_clinical_merged.csv.gz")
gene_file = os.path.join(DIRS["RAW"], "gene_annotation.csv.gz")

# Vérification de l'existence et chargement des données brutes
if os.path.exists(expr_file) and os.path.exists(meta_file) and os.path.exists(gene_file):
    print("Lecture des fichiers compressés (cela peut prendre quelques secondes)...")
    
    # Lecture avec pandas (gestion native du .gz)
    # index_col=0 car la première colonne contient les barcodes/identifiants
    df_expr = pd.read_csv(expr_file, index_col=0)
    df_meta = pd.read_csv(meta_file, index_col=0)
    df_gene = pd.read_csv(gene_file, index_col=0)
    
    print(f"\n✅ Vérification terminée avec succès.")
    print(f"   - Matrice de Counts  ({os.path.basename(expr_file)})  : {df_expr.shape[0]} échantillons x {df_expr.shape[1]} gènes")
    print(f"   - Métadonnées        ({os.path.basename(meta_file)})  : {df_meta.shape[0]} échantillons x {df_meta.shape[1]} colonnes cliniques")
    print(f"   - Informations Gènes ({os.path.basename(gene_file)})  : {df_gene.shape[0]} gènes x {df_gene.shape[1]} colonnes annotées\n")
    
    # Affichage des premières lignes pour inspection visuelle
    display(df_expr.head(3))
    display(df_meta.head(3))
    display(df_gene.head(3))

else:
    print(f"❌ Erreur : Fichiers introuvables dans {DIRS['RAW']}.")
    print("Veuillez vérifier que la cellule 2Bis (Relocalisation) a bien été exécutée.")

Lecture des fichiers compressés (cela peut prendre quelques secondes)...

✅ Vérification terminée avec succès.
   - Matrice de Counts  (counts_samples_x_genes.csv.gz)  : 553 échantillons x 60660 gènes
   - Métadonnées        (metadata_clinical_merged.csv.gz)  : 553 échantillons x 175 colonnes cliniques
   - Informations Gènes (gene_annotation.csv.gz)  : 60660 gènes x 2 colonnes annotées



,ENSG00000000003.15,ENSG00000000005.6,ENSG00000000419.13,ENSG00000000457.14,ENSG00000000460.17,ENSG00000000938.13,ENSG00000000971.16,ENSG00000001036.14,ENSG00000001084.13,ENSG00000001167.14,...,ENSG00000288661.1,ENSG00000288662.1,ENSG00000288663.1,ENSG00000288665.1,ENSG00000288667.1,ENSG00000288669.1,ENSG00000288670.1,ENSG00000288671.1,ENSG00000288674.1,ENSG00000288675.1
sample_barcode,,,,,,,,,,,,,,,,,,,,,
TCGA-EY-A1GK-01A-11R-A13S-07,1904,2,607,529,138,167,238,1427,597,877,...,0,1,20,0,0,0,68,0,2,44
TCGA-FI-A2CX-01A-11R-A17B-07,496,1,839,253,194,4301,847,3321,2903,1147,...,0,1,18,0,0,3,85,0,9,7
TCGA-AX-A3G3-01A-11R-A213-07,2115,2,2657,352,284,673,952,1760,1092,1317,...,0,0,7,0,0,2,101,0,13,129


,patient,sample,shortLetterCode,definition,sample_submitter_id,tumor_descriptor,sample_id,pathology_report_uuid,submitter_id,sample_type,...,treatments_radiation_treatment_dose,treatments_radiation_number_of_fractions,treatments_radiation_treatment_anatomic_sites,treatments_radiation_treatment_dose_units,treatments_radiation_prescribed_dose_units,treatments_radiation_route_of_administration,treatments_radiation_number_of_cycles,treatments_radiation_prescribed_dose,treatments_radiation_residual_disease,bcr_patient_barcode.y
barcode,,,,,,,,,,,,,,,,,,,,,
TCGA-EY-A1GK-01A-11R-A13S-07,TCGA-EY-A1GK,TCGA-EY-A1GK-01A,TP,Primary solid Tumor,TCGA-EY-A1GK-01A,Primary,d4e702a9-8e4a-4ad7-8b45-691c95dcf298,B5024D10-E23D-4B3B-8C54-13EBAA423A04,TCGA-EY-A1GK,Primary Tumor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-EY-A1GK
TCGA-FI-A2CX-01A-11R-A17B-07,TCGA-FI-A2CX,TCGA-FI-A2CX-01A,TP,Primary solid Tumor,TCGA-FI-A2CX-01A,Primary,b6d24830-0d69-44d7-94cf-7dfdc49e52c4,0F6F56E4-C4F9-4915-BE4E-1FBF4C6D93AD,TCGA-FI-A2CX,Primary Tumor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-FI-A2CX
TCGA-AX-A3G3-01A-11R-A213-07,TCGA-AX-A3G3,TCGA-AX-A3G3-01A,TP,Primary solid Tumor,TCGA-AX-A3G3-01A,Primary,5efdace8-5790-4239-8e8a-6349f88b10dd,3D86258F-2C9B-4DF2-86C5-E12B3FCB480D,TCGA-AX-A3G3,Primary Tumor,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TCGA-AX-A3G3


,gene_symbol,gene_type
ensembl_gene_id,,
ENSG00000000003.15,TSPAN6,protein_coding
ENSG00000000005.6,TNMD,protein_coding
ENSG00000000419.13,DPM1,protein_coding


### Guide de Référence : Les Barcodes TCGA
<Small>
L'identifiant TCGA (Barcode) est la clé de voûte de l'organisation des données. Il permet de remonter de la donnée brute (séquençage) jusqu'à l'entité biologique (le patient).

1. Le Barcode Long (Échantillon / Sample)
Ce code complet est utilisé dans la Matrice d'Expression. Il détaille précisément l'origine technique de la donnée.
<Br>Projet (TCGA) : The Cancer Genome Atlas.
<Br>TSS (EY) : Tissue Source Site (le centre où le prélèvement a été effectué).
<Br>Participant (A1GK) : ID Patient (identifiant unique de la patiente, clé à 12 caractères).
<Br>Sample (01) : Type d'échantillon (01 : tumeur solide primaire, 11 : tissu normal).
<Br>Vial (A) : La fiole spécifique (portion) du prélèvement.
<Br>Portion (11) : La portion physique utilisée pour l'extraction.
<Br>Analyte (R) : Le type de molécule extraite (R pour l'ARN, D pour l'ADN).
<Br>Plate (A13S) : Identifie la plaque de 96 puits utilisée pour le séquençage.
<Br>Center (07) : Le centre de séquençage qui a généré la donnée.

2. Le Barcode Court (Patient / Participant)
Ce code est utilisé comme identifiant dans la Matrice Clinique et sert de référence d'identité.
<Br>Projet (TCGA) : The Cancer Genome Atlas.
<Br>TSS (EY) : Tissue Source Site (Centre de prélèvement).
<Br>Participant (A1GK) : Identifiant unique de la patiente.

Usage Stratégique : La "Common Key"
Le code à 12 caractères (TCGA-EY-A1GK) est la Clé de Jointure universelle. Son rôle est vital pour deux raisons :
- Alignement des Espaces : Il permet d'aligner chaque vecteur de l'Espace Génique (une ligne de la matrice d'expression) avec sa cible correspondante dans l'Espace Clinique (une ligne de la matrice de survie).
- Normalisation : Il permet de faire abstraction des variables techniques (fioles, plaques, centres) pour ne conserver que la signature biologique liée à l'individu.
- Règle de traitement : Pour fusionner les données, on réduit systématiquement le barcode long au format court (les 12 premiers caractères) afin de faire correspondre les gènes aux données cliniques.
</Small>

## 4. Prochaines étapes
- Nettoyage des noms de gènes (Mapping ENSG -> Symbol).
- Filtrage de l'expression faible (Low counts).
- Normalisation Log-CPM.


In [ ]:
import os
import pandas as pd
from datetime import datetime

path_raw = DIRS["RAW"]
f_counts = os.path.join(path_raw, "counts_samples_x_genes.csv.gz")
f_meta = os.path.join(path_raw, "metadata_clinical_merged.csv.gz")
f_anno = os.path.join(path_raw, "gene_annotation.csv.gz")

df_expr = pd.read_csv(f_counts, index_col=0, compression='gzip')
df_meta = pd.read_csv(f_meta, index_col=0, compression='gzip')
df_gene = pd.read_csv(f_anno, index_col=0, compression='gzip')

mtime_expr = datetime.fromtimestamp(os.path.getmtime(f_counts)).strftime('%Y-%m-%d %H:%M')
mtime_meta = datetime.fromtimestamp(os.path.getmtime(f_meta)).strftime('%Y-%m-%d %H:%M')
mtime_anno = datetime.fromtimestamp(os.path.getmtime(f_anno)).strftime('%Y-%m-%d %H:%M')

size_expr = os.path.getsize(f_counts)
size_meta = os.path.getsize(f_meta)
size_anno = os.path.getsize(f_anno)

print("PIPELINE 0A (ACQUISITION) STATUS REPORT : ")
print("\n✅ SUMMARY :")
print("- Acquisition des donnees brutes via TCGAbiolinks (R).")
print("- Relocalisation des fichiers dans data/raw.")

print("\n✅ DATASET LOCATION : data/raw/")
print(f"    - counts_samples_x_genes.csv.gz     | Modifie : {mtime_expr} | Taille : {size_expr} octets")
print(f"    - metadata_clinical_merged.csv.gz   | Modifie : {mtime_meta} | Taille : {size_meta} octets")
print(f"    - gene_annotation.csv.gz            | Modifie : {mtime_anno} | Taille : {size_anno} octets")

print(f"\n✅ Donnees chargees :")
print(f"    - df_expr : {df_expr.shape[0]} echantillons x {df_expr.shape[1]} genes")
print(f"    - df_meta : {df_meta.shape[0]} echantillons x {df_meta.shape[1]} colonnes")
print(f"    - df_gene : {df_gene.shape[0]} genes x {df_gene.shape[1]} colonnes")

PIPELINE 0A (ACQUISITION) STATUS REPORT : 

✅ SUMMARY :
- Acquisition des donnees brutes via TCGAbiolinks (R).
- Relocalisation des fichiers dans data/raw.

✅ DATASET LOCATION : data/raw/
    - counts_samples_x_genes.csv.gz     | Modifie : 2025-12-31 13:12 | Taille : 28775270 octets
    - metadata_clinical_merged.csv.gz   | Modifie : 2025-12-31 13:12 | Taille : 176175 octets
    - gene_annotation.csv.gz            | Modifie : 2025-12-31 13:12 | Taille : 501961 octets

✅ Donnees chargees :
    - df_expr : 553 echantillons x 60660 genes
    - df_meta : 553 echantillons x 175 colonnes
    - df_gene : 60660 genes x 2 colonnes
